# Sisendtekstide valideerimine ja puhastamine

Tekstide puhastamiseks ja valideerimiseks on mitmeid viise. 
Üks lihtsamaid võimalusi on kõigepealt teha tekstides standardsed asendused ning seejärel kontrollida tulemust vaadates sõnastuse tulemusena tekkivate sõnavormide sagedust ning otsides neist võimalikke erindeid.  


In [1]:
import re
import math
import time
import requests
import datetime
import urllib.parse

from datetime import date
from pandas import DataFrame 
from pandas import read_csv
from pandas import concat
from tqdm.auto import tqdm

## I. Tekstidele rakendatavad standardsed normaliseerimisvõtted

Asendame kõik standardsed sõnaeraldajad tühikuga ning eraldame kirjavahemärgid mõlemalt poolt tühikuga.
Kõik standardsed loomuliku keele töötluse vahendid teevad rohkem analoogseid teisendusi. 
Meie eesmärk siin on teha need asendused lihtsustamaks teksti, et muude vigade tuvastamine oleks lihtsam.  

In [2]:
sources = {}
sources['state_laws'] = read_csv('../results/source_texts/state_laws.csv', header=0)
sources['government_regulations'] = read_csv('../results/source_texts/government_regulations.csv', header=0)
sources['local_government_acts'] = read_csv('../results/source_texts/local_government_acts.csv', header=0)
sources['government_orders'] = read_csv('../results/source_texts/government_orders.csv', header=0)

In [3]:
PUNCTUATION_MARK = '[\.,:;!?]'
PUNCTUATION_MARK_GROUP = f'(?P<mark>{PUNCTUATION_MARK})'
PUNCTUATION_MARK_REPLACEMENT = r' \g<mark> '
assert re.sub(PUNCTUATION_MARK_GROUP, PUNCTUATION_MARK_REPLACEMENT, 'male,kabe') == 'male , kabe'

In [4]:
for name, tbl in sources.items(): 
    tbl['document_title'] = tbl['document_title'].str.replace(PUNCTUATION_MARK_GROUP, PUNCTUATION_MARK_REPLACEMENT, regex=True)
    tbl['document_title'] = tbl['document_title'].str.replace('\s+', ' ', regex=True)

## II. Vabamorfi kasutamine sõnade algvormi leidmiseks

Tekstide edasisel analüüsimisel saame kasutada Vabamorfi veebiteennust, mis leiab igale sõnale vastava algvormi.
Kuna mõnele sõnavormile vastab mitu erinevat algvormi (`sadama` -> `sadam`, `sadama`) siis on oluline teada, kas
Vabamorf kasutab kõrvalolevate sõnade konteksti tuvastamaks milline neist võimalikest algvormidest on õige.
Kuna see protsess pole alati veavaba, siis antud analüüsis kasutame kõiki sõnale vastavaid algvorme.
Vabamorfi teiseks eripäraks on oletamine. Vabamorf kasutab sisemiselt suurt sõnastikku algvormide määramiseks.
Kui oletamine on sisse lülitatud, siis kasutatakse heuristilisi reegleid algvormide leidmiseks sõnadele, mis pole suures sõnastikus.
Oletamise kasutamine on suurte tekstide analüüsimisel vajalik, et saaks korrektselt käsitleda erialatermineid ja slängi. 

**Tehniline märkus.** Sõnade lemmatiseerimiseks saab kasutada kahte erinevat veebiteenust 
* sõnade morfoloogilist analüsaatorit (`analyser`)
* dokumentide indekseerijat (`indekseerija_lemmad`)

Neist esimene analüüsib sisendit sõnade haaval ning teine analüüsib kogu sisendit korraga ning ühestab mitmesed analüüsid lähtuvalt kontekstist. Antud juhul sobib meile sõnahaaval tehtud analüüs, kuna me valideerime sisendi korrektsust.  

In [5]:
def analyze_text(caption: str):
    """
    Uses web service to extract words and sub-words form document captions

    Returns a four column table with columns index, wordform, lemma, sublemmas.
    There can be several rows for each word as each word is analysed separately.
    All rows with the same index correspond to the same word.
    Wordform columns is added to facilitate tokenisation debugging.
    """
    ANALYZER_QUERY = "https://smart-search.tartunlp.ai/api/analyser/process"
    HEADERS = {"Content-Type": "application/json; charset=utf-8"}
    POST_DATA_TEMPLATE = {'params': {"vmetajson": ["--guess"]}, 'content': caption}

    response = requests.post(ANALYZER_QUERY, json=POST_DATA_TEMPLATE, headers=HEADERS)
    assert response.ok, "Webservice failed"
    response = response.json()

    token_count = len(response['annotations']['tokens'])
    tbl = DataFrame({'wordform': [None] * token_count, 'lemma': [None] * token_count})
    for i, token in enumerate(response['annotations']['tokens']):
        features = token['features']
        tbl.loc[i, 'wordform'] = features['token']
        tbl.loc[i, 'lemma'] = list(set(map(lambda x: x['lemma'], features['mrf'])))

    tbl =  tbl.reset_index().explode('lemma')

    # Post-correction for Vabamorph output. Remove special symbols 
    tbl['lemma'] = tbl['lemma'].str.replace('=', '', regex=False)
    tbl['lemma'] = tbl['lemma'].str.replace('+', '', regex=False)

    # Post-correction for sublemmas
    tbl['sublemmas'] = tbl['lemma'].str.split('_', regex=False)
    tbl['lemma'] = tbl['lemma'].str.replace('_', '', regex=False)
    return tbl

# Example output
analyze_text('Presidendi ametiraha seadus')

,index,wordform,lemma,sublemmas
0,0,Presidendi,president,[president]
1,1,ametiraha,ametiraha,"[ameti, raha]"
2,2,seadus,seadu,[seadu]
2,2,seadus,seadus,[seadus]


In [6]:
all_documents = concat([source for _,source in sources.items()], axis=0)

In [7]:
result = [None] *  len(all_documents)
for i, caption in tqdm(enumerate(all_documents['document_title']), total=len(all_documents)):
    result[i] = analyze_text(caption).assign(doc_id = i)

result = concat(result, axis=0)
display(result)

  0%|          | 0/58837 [00:00<?, ?it/s]

ConnectionError: ('Connection aborted.', TimeoutError(60, 'Operation timed out'))

## III. Sisendtekstide analüüs

Kõige lihtsam on anomaalilad tuvastada analüüsides esmalt sõna algvorme (lemmasid).
Kõigepealt on kasulik otsida anomaaliaid harva esinevate algvormide hulgast.

In [ ]:
lemma_counts = (result
                .groupby('lemma').agg(occurence_count=('lemma', len), document_count = ('doc_id', lambda x: len(set(x))))
                .sort_values(['occurence_count', 'document_count'], ascending=False).reset_index()
                .pipe(lambda df: df[df['lemma'].str.len() > 0])
               )
display(lemma_counts.tail(15))

**Tulemus:** Harva esinevate lemmade hulgas on tõesti palju anomaalseid sõnu. 

In [ ]:
ESTONIAN_LETTER = '[a-z|öäõü|\-|žš]'
ESTONIAN_LETTERS_ONLY = f'^(?:{ESTONIAN_LETTER})+$' 

Vaatame neid erandlikke sõnu lähemalt jättes alles kõik need sõnad, mis sisaldavad eesti tähestikku mitte kuuluvaid sümboleid.

In [ ]:
incorrect = ~lemma_counts['lemma'].str.contains(ESTONIAN_LETTERS_ONLY, case=False, regex=True)
display(concat([
    lemma_counts[incorrect]
    .head(15)
    .reset_index(drop=True),
    lemma_counts[incorrect]
    .tail(15)
    .reset_index(drop=True)], axis=1))

**Tulemus:** Lemmade hulgas on palju spetsiifilisi sõnu: kirjavahemärke ja aastanumbreid.


In [ ]:
NUMBER_SYMBOLS = '[0-9]'
NUMBERS_ONLY = f'^{NUMBER_SYMBOLS}+$'
numbers = lemma_counts['lemma'].str.contains(NUMBERS_ONLY, case=False, regex=True)
unanalysed = incorrect & ~ numbers
display(lemma_counts[unanalysed & lemma_counts['lemma'].str.contains(f'{NUMBER_SYMBOLS}+')])

**Tulemus:** Lisaks numbritele on meil ka numbrivahemikud

In [ ]:
DASH_SYMBOLS = '[-−‒]'
SLASH_SYMBOLS = '[/]'
NUMBER = f'{NUMBER_SYMBOLS}+'
NUMBER_RANGE = f'(?:(?:{NUMBER})?{DASH_SYMBOLS}{NUMBER})|(?:{NUMBER}{DASH_SYMBOLS})|(?:{NUMBER}{SLASH_SYMBOLS}{NUMBER})'
NUMBER_RANGE_ONLY = f'^(?:{NUMBER_RANGE})$'
number_range = lemma_counts['lemma'].str.contains(NUMBER_RANGE_ONLY)
assert re.match(NUMBER_RANGE_ONLY, '-2012') is not None
assert re.match(NUMBER_RANGE_ONLY, '1930-2023') is not None
assert re.match(NUMBER_RANGE_ONLY, '2017/2018') is not None
assert re.match(NUMBER_RANGE_ONLY, '−2025') is not None
assert re.match(NUMBER_RANGE_ONLY, '2017-') is not None
assert re.match(NUMBER_RANGE_ONLY, '2014‒2020') is not None
unanalysed = incorrect & ~ numbers & ~number_range 
display(lemma_counts[unanalysed & lemma_counts['lemma'].str.contains(NUMBER)])

**Tulemus:** Lisaks numbrivahemikele on olemas ka protsendid ja kirjavahemärgid

In [ ]:
PERCENT_SYMBOLS ='[%]'
PERCENT_ONLY = f'^(?:{NUMBER}{PERCENT_SYMBOLS})$'
PUNCTUATION_MARK_ONLY =f'^(?:{PUNCTUATION_MARK})$'
percents = lemma_counts['lemma'].str.contains(PERCENT_ONLY)
punctuation_marks = lemma_counts['lemma'].str.contains(PUNCTUATION_MARK_ONLY)
unanalysed = incorrect & ~ numbers & ~number_range & ~percents & ~punctuation_marks 
display(lemma_counts[unanalysed & lemma_counts['lemma'].str.contains(NUMBER)])

**Tulemus:** Jutumärgid on jäänud kirjavahemärkide hulgast välja.

In [ ]:
PUNCTUATION_MARK = '[\.,:;!?"ˮ“\)\(]'
PUNCTUATION_MARK_GROUP = f'(?P<mark>{PUNCTUATION_MARK})'
PUNCTUATION_MARK_REPLACEMENT = r' \g<mark> '
assert re.sub(PUNCTUATION_MARK_GROUP, PUNCTUATION_MARK_REPLACEMENT, 'male,kabe') == 'male , kabe'
assert re.sub(PUNCTUATION_MARK_GROUP, PUNCTUATION_MARK_REPLACEMENT, '2010ˮ') == '2010 ˮ '
assert re.sub(PUNCTUATION_MARK_GROUP, PUNCTUATION_MARK_REPLACEMENT, '(2010)') == ' ( 2010 ) '
punctuation = lemma_counts['lemma'].str.contains(f'^{PUNCTUATION_MARK}$')
display(lemma_counts[punctuation].style.set_caption('Kirjavahemärgid'))
unanalysed &= ~punctuation
display(lemma_counts[unanalysed].head().style.set_caption('Veel liigitamata sõnad'))
display(lemma_counts[unanalysed & (lemma_counts['lemma'].str.len() == 1)].style.set_caption('Ühetähelised liigitamata sõnad'))

**Tulemus:** Lisaks punktuatsioonile on veel sümboleid, mida peaks ignoreerima.

Vaatame nüüd kahetähelisi ja kolmetähelisi sõnu, sest need sisaldavad ilmsesti ootamatuid sümboleid.  

In [ ]:
OTHER_SYMBOLS = '[§/−a]'
OTHER_SYMBOLS_ONLY=f'^(?:{OTHER_SYMBOLS})$'
other_symbols = lemma_counts['lemma'].str.contains(OTHER_SYMBOLS_ONLY)
unanalysed &= ~ other_symbols
display(lemma_counts[unanalysed].head().style.set_caption('Veel liigitamata sõnad'))
display(lemma_counts[unanalysed & (lemma_counts['lemma'].str.len() == 2)].style.set_caption('Kahetähelised liigitamata sõnad'))
display(lemma_counts[unanalysed & (lemma_counts['lemma'].str.len() == 3)].style.set_caption('Kolmetähelised liigitamata sõnad'))

**Tulemus:** Spetsiaalsed ülaindeksite sümboleid tähistavad sümbolid on problemaatilised.

Uurime seda täpsemalt filtreerides välja kõik sõnad, milles olevaid sümboleid me juba lubame.

In [ ]:
ALLOWED_SYMBOLS = f'{ESTONIAN_LETTER}|{NUMBER_SYMBOLS}|{PUNCTUATION_MARK}|{DASH_SYMBOLS}|{SLASH_SYMBOLS}|{PERCENT_SYMBOLS}|{OTHER_SYMBOLS}'
lemma_counts[unanalysed & ~lemma_counts['lemma'].str.contains(f'^(?:{ALLOWED_SYMBOLS})+$', regex=True, case=False)]

**Tulemus:** Eestikeele tähestikku tuleb veel lisada paar tähte katmaks ära võõrkeelseid nimesid.

In [ ]:
ESTONIAN_LETTER = '[a-z|öäõü|\\-|žš|ôíë]'
ALLOWED_SYMBOLS = f'{ESTONIAN_LETTER}|{NUMBER_SYMBOLS}|{PUNCTUATION_MARK}|{DASH_SYMBOLS}|{SLASH_SYMBOLS}|{PERCENT_SYMBOLS}|{OTHER_SYMBOLS}'
lemma_counts[unanalysed & ~lemma_counts['lemma'].str.contains(f'^(?:{ALLOWED_SYMBOLS})+$', regex=True, case=False)]

**Tulemus:** Paljudes sõnades on unicode superskripti sümbolid. 

In [ ]:
SUBSCRIPT_SYMBOLS = '[₀₁₂₃₄₅₆₇₈₉₊₋]'
SUPERSCRIPT_SYMBOLS = '[⁰¹²³⁴⁵⁶⁷⁸⁹⁺⁻]'
lemma_counts[lemma_counts['lemma'].str.contains(f'{SUPERSCRIPT_SYMBOLS}|{SUBSCRIPT_SYMBOLS}', regex=True)]

In [ ]:
ALLOWED_SYMBOLS = (f'{ESTONIAN_LETTER}|{NUMBER_SYMBOLS}|{PUNCTUATION_MARK}|{DASH_SYMBOLS}' 
                  f'|{SLASH_SYMBOLS}|{PERCENT_SYMBOLS}|{OTHER_SYMBOLS}|{SUPERSCRIPT_SYMBOLS}|{SUBSCRIPT_SYMBOLS}')
lemma_counts[unanalysed & ~lemma_counts['lemma'].str.contains(f'^(?:{ALLOWED_SYMBOLS})+$', regex=True, case=False)]

Leiame nüüd ülesse kõik sümbolid, mis jäävad lubatud sümbolitest välja.

In [ ]:
idx = unanalysed & ~lemma_counts['lemma'].str.contains(f'^(?:{ALLOWED_SYMBOLS})+$', regex=True, case=False)
set.union(*lemma_counts.loc[idx, 'lemma'].map(lambda x: set(x) - set(re.findall(ALLOWED_SYMBOLS, x, flags=re.IGNORECASE))))

* Siit on näha, et me peame uuendama oma kirjavahemärkide definitsiooni lisades sinna erinevaid jutumärke. 
* Kuna ülakomasid kasutatakse käändelõppude eraldamiseks ja me eraldame kirjavahemärgid tühikutega, siis neid me kirjavahemärkide  hulka ei pane.
* Lisaks tuleb meil uuendada kriipsude sümboleid ning defineerida erikujuliste tühikute symbolid ning täiendada eestikeelset tähestikku.

In [ ]:
DASH_SYMBOLS = '[‑-−‒]'
PUNCTUATION_MARK = '[\\.,:;!?¿\\(\\)«»„““ˮ"‟”]'
WHITESPACE_SYMBOLS = '[\u200e\ufeff]'
OTHER_SYMBOLS = '[§/−a\^]'
ESTONIAN_LETTER = '[a-z|öäõü|\\-|žš|ôíëа]'

ALLOWED_SYMBOLS = (f'{ESTONIAN_LETTER}|{NUMBER_SYMBOLS}|{PUNCTUATION_MARK}|{DASH_SYMBOLS}' 
                  f'|{SLASH_SYMBOLS}|{PERCENT_SYMBOLS}|{OTHER_SYMBOLS}|{SUPERSCRIPT_SYMBOLS}|{SUBSCRIPT_SYMBOLS}|{WHITESPACE_SYMBOLS}')
lemma_counts[unanalysed & ~lemma_counts['lemma'].str.contains(f'^(?:{ALLOWED_SYMBOLS})+$', regex=True, case=False)]

## III. Sisendtekstide puhastamine

Eelnevast analüüsist lähtuvalt saame me nüüd defineerida sisendtekstide puhastamise skriptid, mis parandavad esialgset naiivset tokenisatsiooni:
* kõik tühikusümbolid ühestatakse;
* kirjavahemärgid ning muud erisümbolid eraldatakse mõlemalt poolt tühikutega;
* numbrivahemikkude korral eraldame kriipsu numbridümbolitest.

In [ ]:
ESTONIAN_LETTER = '[a-z|öäõüžš]'
FOREIGN_LETTER = '[ôíëаa]'
NUMBER_SYMBOLS = '[0-9]'

# Tühikutega eraldatavad sümbolid 
PUNCTUATION_MARK = '[\\.,:;!?¿\\(\\)«»„““ˮ"‟”]'
SPECIAL_SYMBOLS = '[§/%\^]'
SUPERSCRIPT_SYMBOLS = '[⁰¹²³⁴⁵⁶⁷⁸⁹⁺⁻]'
SUBSCRIPT_SYMBOLS = '[₀₁₂₃₄₅₆₇₈₉₊₋]'

# Tühikutega asendatavad sümbolid  
WHITESPACE_SYMBOLS = '[\u200e\ufeff]'

# Teised lubatud sümbolid
DASH_SYMBOLS = '[‑-−‒-]'
OTHER_SYMBOLS = '[§/−a\^]'

In [ ]:
SEPARATION_GROUP = f'(?P<mark>{PUNCTUATION_MARK}|{SPECIAL_SYMBOLS}|{SUBSCRIPT_SYMBOLS}|{SUPERSCRIPT_SYMBOLS})'
SEPARATION_GROUP_REPLACEMENT = r' \g<mark> '
assert re.sub(SEPARATION_GROUP, SEPARATION_GROUP_REPLACEMENT, 'male,kabe') == 'male , kabe'
assert re.sub(SEPARATION_GROUP, SEPARATION_GROUP_REPLACEMENT, 'male/kabe') == 'male / kabe'
assert re.sub(SEPARATION_GROUP, SEPARATION_GROUP_REPLACEMENT, 'male¹') == 'male ¹ '
assert re.sub(SEPARATION_GROUP, SEPARATION_GROUP_REPLACEMENT, 'male₁') == 'male ₁ '

DASH_GROUP = f'(?P<head>^|\s|{WHITESPACE_SYMBOLS}|{NUMBER_SYMBOLS})(?P<dash>{DASH_SYMBOLS})(?P<tail>\s|{WHITESPACE_SYMBOLS}|{NUMBER_SYMBOLS}|$)'
DASH_GROUP_REPLACEMENT = r'\g<head> \g<dash> \g<tail>'
assert re.sub(DASH_GROUP, DASH_GROUP_REPLACEMENT, '1‑2') == '1 ‑ 2'
assert re.sub(DASH_GROUP, DASH_GROUP_REPLACEMENT, '-2021') == ' - 2021'
assert re.sub(DASH_GROUP, DASH_GROUP_REPLACEMENT, '2021- ') == '2021 -  '

WHITESPACE_GROUP = f'(?:\s|{WHITESPACE_SYMBOLS})+'
assert re.sub(WHITESPACE_GROUP, ' ', '  ') == ' '
assert re.sub(WHITESPACE_GROUP, ' ', '\t  ') == ' '
assert re.sub(WHITESPACE_GROUP, ' ', '\u200e\ufeff ') == ' '

In [ ]:
sources = {}
sources['state_laws'] = read_csv('../results/source_texts/state_laws.csv', header=0)
sources['government_regulations'] = read_csv('../results/source_texts/government_regulations.csv', header=0)
sources['local_government_acts'] = read_csv('../results/source_texts/local_government_acts.csv', header=0)
sources['government_orders'] = read_csv('../results/source_texts/government_orders.csv', header=0)

In [ ]:
for name, tbl in sources.items(): 
    tbl['document_title'] = tbl['document_title'].str.replace(SEPARATION_GROUP, SEPARATION_GROUP_REPLACEMENT, regex=True)
    tbl['document_title'] = tbl['document_title'].str.replace(DASH_GROUP, DASH_GROUP_REPLACEMENT, regex=True)
    tbl['document_title'] = tbl['document_title'].str.replace(WHITESPACE_GROUP, ' ', regex=True)
    tbl.to_csv(f'../results/cleaned_texts/{name}.csv')

## IV. Puhastatud sisendtekstide valideerimine

In [ ]:
sources = {}
sources['state_laws'] = read_csv('../results/cleaned_texts/state_laws.csv', header=0)
sources['government_regulations'] = read_csv('../results/cleaned_texts/government_regulations.csv', header=0)
sources['local_government_acts'] = read_csv('../results/cleaned_texts/local_government_acts.csv', header=0)
sources['government_orders'] = read_csv('../results/cleaned_texts//government_orders.csv', header=0)

In [ ]:
all_documents = concat([source for _,source in sources.items()], axis=0)

In [ ]:
result = [None] *  len(all_documents)
for i, caption in tqdm(enumerate(all_documents['document_title']), total=len(all_documents)):
    result[i] = analyze_text(caption).assign(doc_id = i)

result = concat(result, axis=0)
display(result)

In [ ]:
lemma_counts = (result
                .groupby('lemma').agg(occurence_count=('lemma', len), document_count = ('doc_id', lambda x: len(set(x))))
                .sort_values(['occurence_count', 'document_count'], ascending=False).reset_index()
                .pipe(lambda df: df[df['lemma'].str.len() > 0])
               )
display(lemma_counts.tail(15))

**Kontroll:** Üheelemendiliste sümbolite hulgas pole tundmatuid sümboleid.

In [ ]:
idx = lemma_counts['lemma'].str.len() == 1
idx &= ~lemma_counts['lemma'].str.contains(f'^(?:{ESTONIAN_LETTER}|{FOREIGN_LETTER}|{NUMBER_SYMBOLS})$', case=False)
idx &= ~lemma_counts['lemma'].str.contains(f'^(?:{PUNCTUATION_MARK}|{SUBSCRIPT_SYMBOLS}|{SUPERSCRIPT_SYMBOLS})$', case=False)
idx &= ~lemma_counts['lemma'].str.contains(f'^(?:{DASH_SYMBOLS}|{SPECIAL_SYMBOLS})$', case=False)
assert not any(idx)

**Filtreerimine:** Eemaldame kõik normaalsete tähetedega lemmad ja numbrid ja õhesümbolised sõnad.

In [ ]:
unanalysed = lemma_counts['lemma'].str.len() != 1

idx = lemma_counts['lemma'].str.contains(f'^(?:{ESTONIAN_LETTER}|{FOREIGN_LETTER})+$', case=False)
unanalysed &= ~ idx
display(lemma_counts[idx].sample(n=10))

idx = lemma_counts['lemma'].str.contains(f'^(?:{NUMBER_SYMBOLS})+$', case=False)
unanalysed &= ~ idx
display(lemma_counts[idx].sample(n=10))

**Filtreerimine:** Eemaldame kõik sidekriipsuga sõnad

In [ ]:
idx = lemma_counts['lemma'].str.contains(f'^(?:{ESTONIAN_LETTER}|{FOREIGN_LETTER}|{DASH_SYMBOLS})+$', case=False)
display(lemma_counts[idx & unanalysed].sample(10))
unanalysed &= ~idx

**Tulemus:** Alles jääb hulk vigaseid sõnu ja lühendeid, mida tokenisatsioon ei peagi parandama:
* superscripti sümbol on lisatud nimisõnale;
* tekstile järgneb sidekriipsuga number;
* valesti kirjutatud sõnad tüüpi 17-aastane;
* lühendid kujul 104-k;
* lühendid kujul 10a ja 9c.

In [ ]:
pattern = f'^(?:{ESTONIAN_LETTER}|{FOREIGN_LETTER}|{DASH_SYMBOLS})*(?:{ESTONIAN_LETTER}|{FOREIGN_LETTER}){NUMBER_SYMBOLS}$'
idx = lemma_counts['lemma'].str.contains(pattern, case=False)
display(lemma_counts[idx])
unanalysed &= ~ idx

pattern = f'^(?:{ESTONIAN_LETTER}|{FOREIGN_LETTER})+{DASH_SYMBOLS}(?:{NUMBER_SYMBOLS})+$'
idx = lemma_counts['lemma'].str.contains(pattern, case=False)
display(lemma_counts[idx])
unanalysed &= ~ idx

pattern = f'^(?:{NUMBER_SYMBOLS})+(?:{ESTONIAN_LETTER}|{FOREIGN_LETTER})*ne$'
idx = lemma_counts['lemma'].str.contains(pattern, case=False)
display(lemma_counts[idx])
unanalysed &= ~idx

pattern = f'^(?:{NUMBER_SYMBOLS})+{DASH_SYMBOLS}{ESTONIAN_LETTER}$'
idx = lemma_counts['lemma'].str.contains(pattern, case=False)
display(lemma_counts[idx])
unanalysed &= ~idx

pattern = f'^(?:{NUMBER_SYMBOLS})+{ESTONIAN_LETTER}$'
idx = lemma_counts['lemma'].str.contains(pattern, case=False)
display(lemma_counts[idx])
unanalysed &= ~idx

**Tulemus:** Allesjäänud lühendid ja sõnad on valiidsed.

In [ ]:
lemma_counts[unanalysed]